### Script for checking reliability of loop calling of hichipper
This script is used to check if the reliability of hichipper increases using peaks called with the new algorithm compared to the included loop calling method.

This will take in input the converted washu interactions format filtered by FDR < 0.10 because at least all interactions appear only once and this makes it easier.
```
Premature optimization is the root of all evil (or at least most of it) in programming
Donald Knuth
```

In [ ]:
from ipyparallel import Client
rc = Client()

In [2]:
%%px
### import necessary functions
import scipy
import scipy.sparse, scipy.stats
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot as plt 
import matplotlib.pyplot
import time
import statistics
import sklearn
import statsmodels , statsmodels.api
import itertools
import random
import math

In [3]:
%%px
def check_overlap(A,B):
    # A and B are two tuples start and end of anchors. need to find if B touches A.
    if (B[0] >= A[0] and B[0] <= A[1]) or (B[1] >= A[0] and B[1] <= A[1]) or (B[0] <= A[0] and B[1] >= A[1]) :
        #which means if one of the 2 ends of B is within A by at least 1 bp then ok
        return True
    else:
        return False
    

def check_presence(database, loop,chromosome):
    #checks if the loop is in the loop database
    for item in database[chromosome]:
        if (check_overlap(item[0],loop[0]) and check_overlap(item[1],loop[1])) or (check_overlap(item[0],loop[1]) and check_overlap(item[1],loop[0])):
            return True
    return False

# database is a list of tuples(loops)
# each loop is a tuple containing the tuple of two anchors
# chromosome is now going to be assumed correct
# [((startA,endA),(startB,endB)),((startA,endA),(startB,endB)),....]

In [4]:
%%px
def calculate_overlap(reference_database,test_database):
    # check how many things overlap between control and test.
    overlapping = []
    for chromosome in test_database:
        print(f"currently searching in {chromosome}")
        for item_test in test_database[chromosome]:
            overlapping.append(check_presence(reference_database,item_test,chromosome))
    return sum(overlapping),len(overlapping)



In [5]:
%%px
def create__test_database(fileobj):
    # takes the washu format file object and creates a database
    database = []
    for line in fileobj:
        data = line.split("\t")
        start = data[0].split(",")
        end = data[1].split(",")
        database.append(((start[0],int(start[1]),int(start[2])),(end[0],int(end[1]),int(end[2]))))
    return database

def create_database(fileobj):
    # takes the washu format file object and creates a database
    database = {}
    for line in fileobj:
        data = line.split("\t")
        start = data[0].split(",")
        end = data[1].split(",")
        if start[0] != end[0]:
            raise Exception("no")
        try:
            database[start[0]].append(((int(start[1]),int(start[2])),(int(end[1]),int(end[2]))))
        except KeyError:
            database[start[0]] = []
            database[start[0]].append(((int(start[1]),int(start[2])),(int(end[1]),int(end[2]))))
    return database

In [6]:
%%px --targets 1 --noblock
file1 = "F:\\psa_functional_genomics\\HiChIP_test\\data\\hichipper_newsoft_peaks_no_pad_full\\mumbach_27ac.washu.new.txt"
file2 = "F:\\psa_functional_genomics\\HiChIP_test\\data\\hichipper_newsoft_peaks_tech_no_pad\\NaiveT_27ac_B2_T1.washu.new.txt"


database1 = create_database(open(file1))
database2 = create_database(open(file2))

<AsyncResult: execute>

In [8]:
%%px --targets 1 --noblock
result = calculate_overlap(database2,database1)

<AsyncResult: execute>

In [7]:
%%px --targets 2 --noblock
file1 = "F:\\psa_functional_genomics\\combined_hichip\\data\\hichipper_self\\mumbach_27ac.washu.new.txt"
file2 = "F:\\psa_functional_genomics\\HiChIP_test\\data\\hichipper_self\\NaiveT_27ac_B2_T1.washu.new.txt"


database1 = create_database(open(file1))
database2 = create_database(open(file2))

<AsyncResult: execute>

In [9]:
%%px --targets 2 --noblock
result = calculate_overlap(database2,database1)

<AsyncResult: execute>

In [42]:
%%px 
try:
    print(result)
except NameError:
    pass


In [ ]:
directory = 
MY_full =
MY_HALF =
MY_QUART =
MY_EIGHTH =

HI_full =
HI_HALF =
HI_QUART =
HI_EIGHTH =

KeyboardInterrupt: 

17035463

223 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)
